In [679]:
import numpy as np
import open3d as o3d
import copy
import math
import itertools
import sys
import matplotlib.pyplot as plt
from collections import Counter

import config
cfg = config.PreLoad() # instantiate

## Loading in Point cloud data

UPDATE:
- P003, P013 work successfully
- P013, P014, P015, P017 also work successfully, but require altering z value bounding box plane; otherwise doesn't work.
- P010, P005, P006 works well, but requires sacrificing small details of feet

- P006

18/09:
- P003, P013, P014 works
- try colour threshold around feet


- work on feet refinement new AUT data
- focus on setting error checking pipeline

for report:
- talk about automatic cleaning w manual cleaning - error measure
- how much error is introduced with automatic cleaning - look at benefits / cons
- look at OSSO: Planck Institute
- discuss the scans that have poor quality
- colour boundary, around the feet


P009 Shape of points (50413, 3)

In [689]:
import os, re ## RENAMING & LOADING IN DATA
path = "./data/"

## renaming files if different to the format P00{i}.ply using regex matching
pattern = r"^P\d+\.ply$"
files = []
for filename in os.listdir(path):
    full_path = os.path.join(path, filename)
    
    if os.path.isfile(full_path) and not re.match(pattern, filename):
        files.append(filename) # listing all non uniform filenames

# renaming files if different to standard format
renamed = cfg.rename(path=path, files=files) 
if renamed:
    for old_name, new_name in renamed:
        print(f"Renamed {old_name} to {new_name}")
    
# loading pcd data
path = "./data/P002.ply"
pcd = cfg.load(path)
print("PLY file loaded.")
print("Shape of points", np.asarray(pcd.points).shape)
print("Shape of colors", np.asarray(pcd.colors).shape)
o3d.visualization.draw_geometries([pcd])

PLY file loaded.
Shape of points (36192, 3)
Shape of colors (36192, 3)


In [690]:
from ipywidgets import (
    interact,
    interactive,
    Button,
    FloatSlider,
    FloatText,
)
from ipywidgets import jslink as link
from IPython.display import display
import ipywidgets as widgets

# holds final state of point cloud
final_pcd = None

pcd_tmp = np.asarray(pcd.points)

min_x, min_y, min_z = pcd_tmp.min(axis=0)
max_x, max_y, max_z = pcd_tmp.max(axis=0)


def create_grid_on_plane(thresh, axis, color):
    lines = []
    colors = []
    for i in np.linspace(-1, 1, 10):  # Adjust these values as per your requirements
        # vertical lines
        start = (
            [i, thresh, -1]
            if axis == "y"
            else ([thresh, i, -1] if axis == "x" else [-1, i, thresh])
        )
        end = (
            [i, thresh, 1]
            if axis == "y"
            else ([thresh, i, 1] if axis == "x" else [1, i, thresh])
        )
        lines.append([start, end])
        colors.append(color)
        # horizontal lines
        start = (
            [-1, thresh, i]
            if axis == "y"
            else ([thresh, -1, i] if axis == "x" else [i, -1, thresh])
        )
        end = (
            [1, thresh, i]
            if axis == "y"
            else ([thresh, 1, i] if axis == "x" else [i, 1, thresh])
        )
        lines.append([start, end])
        colors.append(color)

    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(np.array(lines).reshape(-1, 3)),
        lines=o3d.utility.Vector2iVector(
            np.array([[i, i + 1] for i in range(0, len(lines) * 2, 2)])
        ),
    )

    line_set.colors = o3d.utility.Vector3dVector(colors)
    return line_set

def view_and_adjust_threshold(
    thresh_x=min_x,
    thresh_x_dup=max_x,
    thresh_y=min_y,
    thresh_y_dup=max_y,
    thresh_z=min_z,
):
    global final_pcd

    pcd_tmp = np.asarray(pcd.points)
    pcd_tmpc = np.asarray(pcd.colors)

    indices = (
        ((pcd_tmp[:, 0] > thresh_x) & (pcd_tmp[:, 0] < thresh_x_dup))
        & ((pcd_tmp[:, 1] > thresh_y) & (pcd_tmp[:, 1] < thresh_y_dup))
        & (pcd_tmp[:, 2] > thresh_z)
    )
    pcd_tmp_tmp = pcd_tmp[indices]
    pcd_tmp_tmpc = pcd_tmpc[indices]

    filtered = o3d.geometry.PointCloud()
    filtered.points = o3d.utility.Vector3dVector(pcd_tmp_tmp)
    filtered.colors = o3d.utility.Vector3dVector(pcd_tmp_tmpc)

    final_pcd = filtered

    plane_x = create_grid_on_plane(thresh_x, "x", [1, 0, 0])  # red color for x plane
    plane_y = create_grid_on_plane(thresh_y, "y", [0, 1, 0])  # green color for y plane
    plane_z = create_grid_on_plane(thresh_z, "z", [0, 0, 1])  # blue color for z plane

    plane_x_dup = create_grid_on_plane(
        thresh_x_dup, "x", [1, 0, 0]
    )  
    plane_y_dup = create_grid_on_plane(
        thresh_y_dup, "y", [0, 1, 0]
    )  
    
    centroid = np.mean(np.asarray(pcd.points), axis=0)
    centroid_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1, origin=centroid)
    centroid_frame_colored = set_mesh_color(centroid_frame, [1, 0, 0])  # Red color

    ## visualise origin pt
    origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1, origin=[0, 0, 0])
    origin_colored = set_mesh_color(origin, [0, 1, 0])  # Green color

    o3d.visualization.draw_geometries(
        [final_pcd, origin_colored, centroid_frame_colored, plane_x, plane_y, plane_z, plane_x_dup, plane_y_dup]
    )
    
def set_mesh_color(mesh, color):
    """
    Set the color of a TriangleMesh.
    
    :param mesh: TriangleMesh object.
    :param color: List of RGB values. E.g., [1, 0, 0] for red.
    :return: Colored TriangleMesh.
    """
    num_of_vertices = len(np.asarray(mesh.vertices))
    colors = [color for _ in range(num_of_vertices)]
    mesh.vertex_colors = o3d.utility.Vector3dVector(colors)
    return mesh

def save_pcd(b):
    if final_pcd is not None:
        o3d.io.write_point_cloud("final.ply", final_pcd)
        print("Point cloud saved!")
    else:
        print("No point cloud to save.")

save_button = Button(description="Save point cloud")
save_button.on_click(save_pcd)

slider_x = FloatSlider(min=min_x, max=max_x, step=0.001, value=-0.8, description="x")
text_x = FloatText(value=min_x, description="x")
link((slider_x, "value"), (text_x, "value"))

slider_x_dup = FloatSlider(
    min=min_x, max=max_x, step=0.001, value=0.8, description="x_dup"
)
text_x_dup = FloatText(value=max_x, description="x_dup")
link((slider_x_dup, "value"), (text_x_dup, "value"))

slider_y = FloatSlider(min=min_y, max=max_y, step=0.001, value=-0.8, description="y")
text_y = FloatText(value=min_y, description="y")
link((slider_y, "value"), (text_y, "value"))

slider_y_dup = FloatSlider(
    min=min_y, max=max_y, step=0.001, value=0.8, description="y_dup"
)
text_y_dup = FloatText(value=max_y, description="y_dup")
link((slider_y_dup, "value"), (text_y_dup, "value"))

slider_z = FloatSlider(min=min_z, max=max_z, step=0.001, value=0.025, description="z")
text_z = FloatText(value=min_z, description="z")
link((slider_z, "value"), (text_z, "value"))

interactive(
    view_and_adjust_threshold,
    thresh_x=slider_x,
    thresh_x_dup=slider_x_dup,
    thresh_y=slider_y,
    thresh_y_dup=slider_y_dup,
    thresh_z=slider_z,
)

display(text_x, text_x_dup, text_y, text_y_dup, text_z, interact, save_button)

pcd = copy.deepcopy(final_pcd)

FloatText(value=-1.4482367038726807, description='x')

FloatText(value=1.4017632007598877, description='x_dup')

FloatText(value=-1.38722562789917, description='y')

FloatText(value=1.3268368244171143, description='y_dup')

FloatText(value=-0.05450461432337761, description='z')

Button(description='Save point cloud', style=ButtonStyle())

In [691]:
def remove_floor_by_plane_fitting(pcd: o3d.cpu.pybind.geometry.PointCloud, distance_threshold=0.003) -> o3d.cpu.pybind.geometry.PointCloud:
    # Use RANSAC plane segmentation
    plane_model, inliers = pcd.segment_plane(distance_threshold=distance_threshold,
                                             ransac_n=3,
                                             num_iterations=1000)
    [a, b, c, d] = plane_model
    print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

    # Remove points that belong to the plane (i.e., the floor)
    pcd_without_floor = pcd.select_by_index(inliers, invert=True)
    # pcd_without_floor = pcd.select_by_index(inliers, invert=False)
    
    return pcd_without_floor

# Segment and remove the floor using plane fitting
pcd = remove_floor_by_plane_fitting(pcd)
o3d.visualization.draw_geometries([pcd])

Plane equation: -0.00x + -0.00y + 1.00z + -0.03 = 0


In [692]:
# def get_clusters_closest_to_origin(clusters, num_clusters=1):
#     """
#     Get the cluster(s) closest to the origin.
    
#     Parameters:
#     - clusters: List of numpy arrays containing cluster points.
#     - num_clusters: Number of closest clusters to return.
    
#     Returns:
#     - closest_clusters: List of numpy arrays containing the closest cluster points.
#     """
    
#     # Compute the centroids of the clusters
#     centroids = [np.mean(cluster, axis=0) for cluster in clusters]
    
#     # Compute the distance of each centroid to the origin
#     distances_to_origin = [np.linalg.norm(centroid) for centroid in centroids]
    
#     # Get the indices of the closest cluster(s) to the origin
#     closest_clusters_idx = np.argsort(distances_to_origin)[:num_clusters]
    
#     # Extract the closest cluster(s)
#     closest_clusters = [clusters[idx] for idx in closest_clusters_idx]
    
#     return closest_clusters


# def visualize_closest_clusters(pcd, clusters, num_clusters=1):
#     """
#     Visualize the cluster(s) closest to the origin in the point cloud with different colors.
    
#     Parameters:
#     - pcd: Original point cloud.
#     - clusters: List of numpy arrays containing cluster points.
#     - num_clusters: Number of closest clusters to visualize.
    
#     Returns:
#     - colored_pcd: Open3D PointCloud object with colored clusters.
#     """
    
#     # Compute the centroids of the clusters
#     centroids = [np.mean(cluster, axis=0) for cluster in clusters]
    
#     # Compute the distance of each centroid to the origin
#     distances_to_origin = [np.linalg.norm(centroid) for centroid in centroids]
    
#     # Get the indices of the closest cluster(s) to the origin
#     closest_clusters_idx = np.argsort(distances_to_origin)[:num_clusters]
    
#     # Create a copy of the original point cloud to color the clusters
#     colors = np.asarray(pcd.colors)
    
#     # Define a set of colors for visualization
#     cluster_colors = [[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 1, 0], [1, 0, 1], [0, 1, 1]]
    
#     # Assign colors to the closest cluster(s)
#     for idx in closest_clusters_idx:
#         cluster_color = cluster_colors[idx % len(cluster_colors)]
#         for point in clusters[idx]:
#             # Find the index of the point in the original point cloud
#             point_idx = np.where((np.asarray(pcd.points) == point).all(axis=1))[0]
#             colors[point_idx] = cluster_color
            
#     pcd.colors = o3d.utility.Vector3dVector(colors)
    
#     return pcd

# Apply the function on the vertices to identify potential feet clusters


# # Get the cluster(s) closest to the origin
# closest_clusters = get_clusters_closest_to_origin(feet_clusters, num_clusters=10)

# # Visualize the cluster(s) closest to the origin
# colored_pcd_closest = visualize_closest_clusters(pcd, closest_clusters, num_clusters=10)
# o3d.visualization.draw_geometries([colored_pcd_closest])

In [693]:
# import sklearn.cluster as cluster

# def get_top_clusters(clusters, alpha=1.0, top_n=2):
#     """
#     Get the top clusters based on highest point count and closest distance to the origin.
    
#     Parameters:
#     - clusters: List of numpy arrays containing cluster points.
#     - alpha: Weighting factor for the distance in the score calculation.
#     - top_n: Number of top clusters to return.
    
#     Returns:
#     - top_clusters: List of numpy arrays containing the points of the top clusters.
#     """
    
#     # Initialize variables to keep track of the top clusters
#     top_scores = [-float('inf')] * top_n
#     top_clusters = [None] * top_n
    
#     # Compute the centroids and sizes of the clusters
#     for cluster in clusters:
#         centroid = np.mean(cluster, axis=0)
#         size = len(cluster)
#         distance_to_origin = np.linalg.norm(centroid)
        
#         # Calculate the score for the cluster
#         score = size - alpha * distance_to_origin
        
#         # Update the top clusters if this cluster has a higher score
#         for i in range(top_n):
#             if score > top_scores[i]:
#                 # Shift lower-ranking clusters down and insert the new one
#                 for j in range(top_n - 1, i, -1):
#                     top_scores[j] = top_scores[j-1]
#                     top_clusters[j] = top_clusters[j-1]
                
#                 top_scores[i] = score
#                 top_clusters[i] = cluster
#                 break
                
#     return top_clusters

# def visualize_top_clusters(pcd, top_clusters):
#     """
#     Visualize the top clusters in the point cloud with different colors.
    
#     Parameters:
#     - pcd: Original point cloud (Open3D PointCloud object).
#     - top_clusters: List of numpy arrays, each containing the points of a top cluster.
    
#     Returns:
#     - None (Displays the visualization)
#     """
    
#     # Create a copy of the original point cloud to color the top clusters
#     colors = np.asarray(pcd.colors)
    
#     # Define a set of unique colors for the top clusters
#     top_cluster_colors = [[1, 0, 0], [0, 1, 0]]
    
#     for idx, cluster in enumerate(top_clusters):
#         cluster_color = top_cluster_colors[idx % len(top_cluster_colors)]
#         for point in cluster:
#             # Find the index of the point in the original point cloud
#             point_idx = np.where((np.asarray(pcd.points) == point).all(axis=1))[0]
#             colors[point_idx] = cluster_color
            
#     pcd.colors = o3d.utility.Vector3dVector(colors)
    
#     # Visualize the point cloud with the top clusters colored differently
#     o3d.visualization.draw_geometries([pcd])

# def identify_feet_clusters(points, z_threshold=0.1, eps=0.02, min_samples=50):
#     """
#     Identify clusters likely to be feet based on height thresholding and clustering.
    
#     Parameters:
#     - points: The point cloud as a numpy array.
#     - z_threshold: Maximum height above the minimum z value to consider for feet clusters.
#     - eps: The maximum distance between two samples for one to be considered as in the neighborhood of the other.
#     - min_samples: The number of samples in a neighborhood for a point to be considered as a core point.
    
#     Returns:
#     - feet_clusters: A list of arrays, each array containing points of a cluster likely to be a foot.
#     """
#     points = np.asarray(pcd.points)    
    
#     # Height-based thresholding
#     min_z = np.min(points[:, 2])
#     potential_feet_points = points[(points[:, 2] <= min_z + z_threshold) & (points[:, 2] > min_z)]
    
#     # DBSCAN clustering
#     db = cluster.DBSCAN(eps=eps, min_samples=min_samples, metric='euclidean').fit(potential_feet_points)
#     labels = db.labels_
    
#     # Extracting clusters
#     feet_clusters = []
#     unique_labels = np.unique(labels)
#     for l in unique_labels:
#         if l != -1:  # Ignore the noise label (-1)
#             cluster_points = potential_feet_points[labels == l]
#             feet_clusters.append(cluster_points)
    
#     return feet_clusters

# feet_clusters = identify_feet_clusters(pcd)


# # Get the best cluster based on the highest point count and closest distance to the origin
# top_clusters = get_top_clusters(feet_clusters, alpha=1.0, top_n=2)# Visualize the best cluster
# visualize_top_clusters(pcd, top_clusters)


In [694]:
# pcd_vis = copy.deepcopy(pcd) # need to do this to create a deep copy of pcd

# def viewClustersViaColours(pcd: o3d.cpu.pybind.geometry.PointCloud) -> None:
#     '''
#         Visualising pcd coloured by cluster applying DBSCAN (Density-Based Spatial Clustering of Applications with Noise) algorithm
#         Alter cluster_dbscan() params for different results
#             eps: max dist between two samples for them to be considered same neighbourhood
#             min_points: min number of points required to form a cluster
#     '''
    
#     labels = np.array(pcd.cluster_dbscan(eps=0.03, min_points=3, print_progress=True))
#     max_label = labels.max()
#     print(f"point cloud has {max_label + 1} clusters")
#     colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1)) # avoids div error
#     colors[labels < 0] = 0 # label=-1 indicates noise
#     pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
#     o3d.visualization.draw_geometries([pcd])

# viewClustersViaColours(pcd_vis)

In [695]:
# def remove_floor_by_plane_fitting(pcd: o3d.cpu.pybind.geometry.PointCloud, distance_threshold=0.003) -> o3d.cpu.pybind.geometry.PointCloud:
#     # Use RANSAC plane segmentation
#     plane_model, inliers = pcd.segment_plane(distance_threshold=distance_threshold,
#                                              ransac_n=3,
#                                              num_iterations=1000)
#     [a, b, c, d] = plane_model
#     print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

#     # Remove points that belong to the plane (i.e., the floor)
#     pcd_without_floor = pcd.select_by_index(inliers, invert=True)
#     # pcd_without_floor = pcd.select_by_index(inliers, invert=False)
    
#     return pcd_without_floor

# # Segment and remove the floor using plane fitting
# pcd = remove_floor_by_plane_fitting(pcd)
# o3d.visualization.draw_geometries([pcd])

In [696]:
# pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.5, max_nn=50))

# # z values
# z_values = np.asarray(pcd.points)[:, 2]

# # 90th percentile of the Z values for plane segmentation
# z_90th_percentile = np.percentile(z_values, 55)
# z_min = z_values.min()
# alpha = 0.01
# distance_threshold = alpha * (z_90th_percentile - z_min)

# plane_model, inliers = pcd.segment_plane(distance_threshold=distance_threshold, ransac_n=3, num_iterations=1000)
# ground_z_min = np.asarray(pcd.select_by_index(inliers).points)[:, 2].min() # MIN GROUND PTS

# height_threshold = 0.02  # 2 centimeters

# to_remove = [i for i in inliers if z_values[i] < ground_z_min + height_threshold]

# pcd_tmp = pcd.select_by_index(to_remove, invert=True)

# # Visualize the result
# centroid = np.mean(np.asarray(pcd.points), axis=0)
# centroid_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1, origin=centroid)
# centroid_frame_colored = set_mesh_color(centroid_frame, [1, 0, 0])  # Red color

# ## visualise origin pt
# origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1, origin=[0, 0, 0])
# origin_colored = set_mesh_color(origin, [0, 1, 0])  # Green color

# o3d.visualization.draw_geometries([pcd_tmp, origin_colored, centroid_frame_colored])

In [697]:
# def optimized_region_growing_segmentation(pcd, initial_inliers, height_threshold=0.1, radius=0.001, max_iterations=None):
#     """
#     Optimized region growing segmentation based on initial ground points.
    
#     Parameters:
#     - pcd: The point cloud.
#     - initial_inliers: List of initial ground points (from plane segmentation).
#     - height_threshold: Maximum height difference to consider a point as ground.
#     - radius: Radius for neighbor search.
#     - max_iterations: Maximum number of iterations to run the algorithm.
    
#     Returns:
#     - ground_points: List of all ground points after region growing.
#     """
    
#     # Convert points to numpy array
#     points = np.asarray(pcd.points)
    
#     # Initialize the KD-tree for efficient neighbor search
#     kdtree = o3d.geometry.KDTreeFlann(pcd)
    
#     # Set of all ground points
#     ground_points = set(initial_inliers)
    
#     # Points to be checked
#     to_check = set(initial_inliers)
    
#     iteration = 0
#     while to_check:
#         # Check for max iterations
#         if max_iterations and iteration >= max_iterations:
#             break
#         iteration += 1
        
#         # Take a point from the set
#         point_idx = to_check.pop()
        
#         # Find neighbors of the point within the given radius
#         _, idx_neighbors, _ = kdtree.search_radius_vector_3d(points[point_idx], radius)
        
#         for idx in idx_neighbors:
#             # If the point is close enough in height and hasn't been considered yet
#             if abs(points[idx, 2] - points[point_idx, 2]) < height_threshold and idx not in ground_points:
#                 ground_points.add(idx)
#                 to_check.add(idx)
    
#     return list(ground_points)


# # Pseudo-code for the iterative approach and post-processing

# def iterative_region_growing(pcd, initial_inliers, iterations, max_cluster_size=100):
#     """
#     Perform iterative region growing based on initial ground points and given iterations.
    
#     Parameters:
#     - pcd: The point cloud.
#     - initial_inliers: List of initial ground points (from plane segmentation).
#     - iterations: List of dictionaries with parameters for each iteration.
#     - max_cluster_size: Maximum size for clusters to be considered as non-ground.
    
#     Returns:
#     - final_ground_points: List of all ground points after iterative region growing.
#     """
    
#     all_ground_points = set(initial_inliers)
    
#     for iter_params in iterations:
#         # Extract parameters for this iteration
#         height_threshold = iter_params["height_threshold"]
#         radius = iter_params["radius"]
#         max_iterations = iter_params.get("max_iterations", None)
        
#         # Perform region growing with the current parameters
#         current_ground_points = optimized_region_growing_segmentation(pcd, 
#                                                                       list(all_ground_points), 
#                                                                       height_threshold, 
#                                                                       radius, 
#                                                                       max_iterations)
        
#         all_ground_points.update(current_ground_points)
    
#     # Post-processing: Clustering on the identified ground points
#     ground_pcd = pcd.select_by_index(list(all_ground_points))
#     with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
#         labels = np.array(ground_pcd.cluster_dbscan(eps=0.02, min_points=10, print_progress=True))
        
#     # Find small clusters and retain them
#     for i in range(labels.max() + 1):
#         cluster = np.where(labels == i)
#         if cluster[0].shape[0] < max_cluster_size:
#             all_ground_points.difference_update(cluster[0])
    
#     return list(all_ground_points)

# # Iteration parameters
# iterations = [
#     {"height_threshold": 0.05, "radius": 0.001},
#     {"height_threshold": 0.1, "radius": 0.002},
#     {"height_threshold": 0.2, "radius": 0.005}
# ]

# # Ensure you have the required inliers from your plane segmentation
# initial_inliers = inliers

# # Apply iterative region growing
# final_ground_points = iterative_region_growing(pcd, initial_inliers, iterations)

# # Visualize the result after removing the identified ground
# pcd_without_ground = pcd.select_by_index(final_ground_points, invert=True)
# o3d.visualization.draw_geometries([pcd_without_ground])

# # # Note: The provided code snippet includes calls to functions/variables like "inliers" and visualization steps which 
# # # are not defined here. Make sure to integrate the optimized function into your complete pipeline to see the results.

# # # Use initial plane segmentation to get starting inliers
# # initial_inliers = inliers

# # # Apply region growing segmentation
# # all_ground_points = optimized_region_growing_segmentation(pcd, initial_inliers)

# # # Visualize the result after removing the identified ground
# # pcd_without_ground = pcd.select_by_index(all_ground_points, invert=False)
# # o3d.visualization.draw_geometries([pcd_without_ground])


In [698]:
# def remove_floor_using_color(pcd, floor_color_range):
#     colors = np.asarray(pcd.colors)

#     floor_indices = np.where(
#         (colors[:, 0] >= floor_color_range[0][0]) & (colors[:, 0] <= floor_color_range[1][0]) &
#         (colors[:, 1] >= floor_color_range[0][1]) & (colors[:, 1] <= floor_color_range[1][1]) &
#         (colors[:, 2] >= floor_color_range[0][2]) & (colors[:, 2] <= floor_color_range[1][2])
#     )[0]

#     pcd_without_floor = pcd.select_by_index(floor_indices, invert=True)

#     return pcd_without_floor

# floor_color_range = [(0.3, 0.3, 0.3), (0.7, 0.7, 0.7)]

# pcd_without_floor = remove_floor_using_color(pcd, floor_color_range)

# o3d.visualization.draw_geometries([pcd_without_floor])


In [699]:
# def segment_by_dominant_color(pcd, threshold=0.1):
#     colors = np.asarray(pcd.colors)
    
#     hist, bin_edges = np.histogramdd(colors, bins=(8, 8, 8))
    
#     dominant_color_index = np.unravel_index(hist.argmax(), hist.shape)
#     dominant_color = [(edge[i] + edge[i+1]) / 2 for i, edge in zip(dominant_color_index, bin_edges)]
    
#     mask = np.all(np.abs(colors - dominant_color) < threshold, axis=1)
    
#     segmented_pcd = pcd.select_by_index(np.where(mask)[0])
    
#     return segmented_pcd

# segmented_pcd = segment_by_dominant_color(pcd)

# # indices of segmented pcd
# segmented_indices = np.asarray(segmented_pcd.points).tolist()

# # find respective indices from original pcd
# original_indices = [i for i, v in enumerate(pcd.points) if tuple(v) in set(map(tuple, segmented_indices))]

# # inverse of segment 
# inverse_pcd = pcd.select_by_index(original_indices, invert=True)

# pcd = copy.deepcopy(inverse_pcd)

# o3d.visualization.draw_geometries([pcd])


In [700]:
# def color_by_z_coordinate(pcd: o3d.cpu.pybind.geometry.PointCloud) -> None:
#     # Get the z-coordinates of all points
#     points = np.asarray(pcd.points)
#     z_coordinates = points[:, 2]
    
#     # Get the indices of the 10 smallest z-coordinates
#     bottom_ten_indices = np.argsort(z_coordinates)[:30]
    
#     # Create a new point cloud for the bottom 10 points
#     bottom_ten_points = o3d.geometry.PointCloud()
#     bottom_ten_points.points = o3d.utility.Vector3dVector(points[bottom_ten_indices])
    
#     # Set the color of the bottom 10 points to red
#     colors = np.array([[0, 1, 0]] * len(bottom_ten_indices))  # RGB for red
#     bottom_ten_points.colors = o3d.utility.Vector3dVector(colors)
    
#     # Display the original point cloud and the colored points together
#     o3d.visualization.draw_geometries([pcd, bottom_ten_points], window_name="coloured points")

def bounding_box_minor(pcd: o3d.cpu.pybind.geometry.PointCloud) -> None:
    
    # get the z-coordinates of all points
    points = np.asarray(pcd.points)
    z_coordinates = points[:, 2]
    
    # define a threshold for what is considered "close to the ground"
    ground_threshold = 0.1
    
    origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1, origin=[0, 0, 0])
    origin_colored = set_mesh_color(origin, [0, 1, 0])  # Green color

    # color_by_z_coordinate(pcd)
    
    # check if any points are close to the ground
    if np.any(z_coordinates < ground_threshold):
        
        # run segment_plane to remove the floor
        _, inliers = pcd.segment_plane(distance_threshold=0.01,
                                                 ransac_n=3,
                                                 num_iterations=1000)
        
        # separate the points that belong to the plane and those that don't
        pcd = pcd.select_by_index(inliers, invert=True)
    
        o3d.visualization.draw_geometries([pcd, origin_colored], window_name='np.any pcd')
    
    # set the bounds for cropping
    bounds = [[-math.inf, math.inf], [-math.inf, math.inf], [0, 0.0458]]
    
    # create bounding box
    bounding_box_points = list(itertools.product(*bounds))
    bounding_box = o3d.geometry.AxisAlignedBoundingBox.create_from_points(
        o3d.utility.Vector3dVector(bounding_box_points))

    # Determine which points are inside the bounding box
    is_inside = bounding_box.get_point_indices_within_bounding_box(pcd.points)

    # Select points outside the bounding box by inverting the selection
    pcd_cleaned = pcd.select_by_index(is_inside, invert=True)


    
    return pcd_cleaned

def downsample_clean(pcd: o3d.cpu.pybind.geometry.PointCloud) -> None:
    '''
    Performs downsampling, outlier removal and bounding box removal
    
    Args:
        pcd (o3d.cpu.pybind.geometry.PointCloud): PointCloud object
    
    Returns:
        type: None 
    '''

    # voxel downsampling: reducing overall num of pts
    voxel_size = 0.01
    downsampled = pcd.voxel_down_sample(voxel_size) 
    
    # outlier cleaning
    _, ind = downsampled.remove_statistical_outlier(nb_neighbors=100, std_ratio=2.0)
    cleaned_pcd = downsampled.select_by_index(ind)
    
    # minimal bounding box
    pcd = bounding_box_minor(cleaned_pcd)

    # # paint removal parts
    epsilon = 0.01
    points = np.asarray(pcd.points)
    indices = np.where(np.abs(points[:, 2] < epsilon))[0]
    
    pcd_in_color = pcd.select_by_index(indices)
    pcd_in_color.paint_uniform_color([1,1,0])
    pcd = pcd.select_by_index(indices, invert=True)
    
    o3d.visualization.draw_geometries([pcd])

    return pcd

# bounding_box_minor(pcd)
pcd = downsample_clean(pcd)
o3d.visualization.draw_geometries([pcd], window_name="Bounding box")

In [701]:
def isolateLargestCluster(
    pcd: o3d.cpu.pybind.geometry.PointCloud, labels: np.ndarray
) -> None:
    """
    Uses DBSCAN to group and isolate the largest point cloud

    Args:
        pcd (o3d.cpu.pybind.geometry.PointCloud): PointCloud object
        labels (np.ndarray): Array of clusters identified by labels

    Returns:
        type: o3d.cpu.pybind.geometry.PointCloud
    """

    # finding the label of largest cluster + ignoring noisy points labeled -1
    counts = Counter(labels)
    largest_cluster_label = max(
        counts.items(), key=lambda x: x[1] if x[0] != -1 else -1
    )[0]

    # get all pts / colors of largest cluster
    largest_cluster_points = np.array(pcd.points)[labels == largest_cluster_label]
    largest_cluster_colors = np.array(pcd.colors)[labels == largest_cluster_label]

    # new point cloud from the largest cluster pts w/ colors
    largest_cluster_pcd = o3d.geometry.PointCloud()
    largest_cluster_pcd.points = o3d.utility.Vector3dVector(largest_cluster_points)
    largest_cluster_pcd.colors = o3d.utility.Vector3dVector(largest_cluster_colors)

    return largest_cluster_pcd

labels = np.array(pcd.cluster_dbscan(eps=0.05, min_points=3, print_progress=True))
pcd = isolateLargestCluster(pcd, labels)
o3d.visualization.draw_geometries([pcd])

In [702]:
def custom_draw_geometry(pcd, point_size=5.0):
    """
    Used for viewing pcd w/o sparsity in point cloud due to downsampling
    """
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(pcd)
    render_option = vis.get_render_option()
    render_option.point_size = point_size  # Set the size of the points
    vis.run()
    vis.destroy_window()

custom_draw_geometry(pcd, point_size=10.0)

### Saving Point Cloud

In [703]:
output_file_name = f"./data/lidar-cleaned/cleaned_{path.split('/')[-1][:-4]}.ply"

if pcd is not None:
    o3d.io.write_point_cloud(output_file_name, pcd)
    print("Point cloud saved!")
else:
    print("No point cloud to save.")

Point cloud saved!


In [704]:
path = f"./data/lidar-cleaned/cleaned_{path.split('/')[-1][:-4]}.ply"

pcd = X.load(path)
o3d.visualization.draw_geometries([pcd], window_name='read')

NameError: name 'X' is not defined